In [22]:
import numpy as np
import pandas as pd
import datetime
from ipyparallel import Client

In [9]:
c = Client(profile='mycluster')
v = c[:]

In [23]:
def pick_and_calculate_angle(rd_num):
    
    import numpy as np
    import pandas as pd
    import datetime
    from ipyparallel import Client

    CYCLES = 100000
    
    print('working on round number ' + str(rd_num))
    output_file = '/scratch/andreas/andreas_mixture/results/all_combos' + str(rd_num) + '.txt'
    
    descriptors = pd.read_csv('/scratch/andreas/andreas_mixture/odorDescriptorsFewer.csv')
    descriptors = descriptors.drop(descriptors.columns[0], axis=1)
    
    all_angles =  pd.DataFrame(index=range(CYCLES),columns=['ang','mix1','mix2'])
    
    for i in range(CYCLES):
    
        currentMix = descriptors.sample(20,replace=False)
        mix1_sum = currentMix[0:10].sum()
        mix2_sum = currentMix[10:20].sum()
        cur_angle = np.arccos( np.dot(mix1_sum,mix2_sum) / (np.sqrt(np.square(mix1_sum).sum())  * np.sqrt(np.square(mix2_sum).sum())))
        all_angles['ang'][i] = cur_angle
        all_angles['mix1'][i] = currentMix.index[0:10].tolist()
        all_angles['mix2'][i] = currentMix.index[10:20].tolist()
        
   

    all_angles[['ang','mix1','mix2']].sort('ang').to_csv(output_file,index=False,header=False)

In [24]:
v.map_sync(pick_and_calculate_angle, range(250))